**Bibliotecas**

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import multiprocessing
from gensim import matutils  # utility fnc for pickling, common scipy operations etc
import time

import unicodedata
import string
import textdistance

**Importação e limpeza dos dados**

In [2]:
cols = ['produto', 'ncm']
# cols2 = ['produto' 'ncm']
df = pd.read_csv('C:/Users/higor/OneDrive/Área de Trabalho/db_tcc/cotri-nome-ncm.csv', header = 0, names = cols, engine = 'python', encoding = 'latin1')
df2 = pd.read_csv('C:/Users/higor/OneDrive/Área de Trabalho/db_tcc/nfe-nome-ncm.csv', header = 0, names = cols, engine = 'python', encoding = 'latin1')
df = df.append(df2)

C:\Users\higor\AppData\Local\Temp\ipykernel_15932\196608791.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2)


In [4]:
stop_words = ['','ampgtembalagemcx12', 'ampgtembalagemdz12',
'ampgtembalagemcx48']

In [3]:
# csv_path = "C:/Users/higor/OneDrive/Área de Trabalho/db_tcc/train.csv"
# df = pd.read_csv(csv_path)

# def clean_data(text):
#     text = re.sub(r'[^ \nA-Za-z0-9À-ÖØ-öø-ÿ/]+', '', text)
#     text = re.sub(r'[\\/×\^\]\[÷]', '', text)
#     return text

def clean_data(text):
    try:
        text = unicode(text, 'utf-8')
    except NameError:
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    text = text.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))) 
    # text = text.translate(str.maketrans('', '', re.sub('/','', string.punctuation)))

    # text = re.sub('\s\s+', ' ', text.strip())
    return str(text)

def change_lower(text):
    text = text.lower()
    return text

stopwords_list = stopwords.words("portuguese")
# stopwords_list.append(stop_words)
def remover(text):
    text_tokens = text.split(" ")
    final_list = [word for word in text_tokens if not word in stopwords_list]
    text = ' '.join(final_list)
    return text

def get_w2vdf(df, column_name):
    w2v_df = pd.DataFrame(df[column_name]).values.tolist()
    for i in range(len(w2v_df)):
        w2v_df[i] = re.sub('\s\s+', ' ', w2v_df[i][0].strip()).split(" ")
        # w2v_df[i] =  re.sub('+ ', '', w2v_df[i][0].strip()).split(" ")
        # w2v_df[i] = w2v_df[i][0].split(" ")
    return w2v_df


**Treinamento do modelo**

In [113]:
def train_w2v(w2v_df):
    cores = multiprocessing.cpu_count()
    print("-- Cores:",cores)
    w2v_model = Word2Vec(min_count=2, #before was = 4
                         window=5,
                         vector_size=300, #vector_size or vector
                         alpha=0.03, 
                         min_alpha=0.0007, 
                         sg = 1,
                         workers=cores-1)
    
    print("-- Building vocab\n")
    w2v_model.build_vocab(w2v_df, progress_per=10000)
    print("-- Training the model\n")
    w2v_model.train(w2v_df, total_examples=w2v_model.corpus_count, epochs=1000, report_delay=1)
    return w2v_model

In [4]:
# transform the text to string type, lower case, clean data
# and remove stop words
df["produto"] = df["produto"].astype(str)
df["produto"] = df["produto"].apply(change_lower)
df["produto"] = df["produto"].apply(clean_data)
df["produto"] = df["produto"].apply(remover)

# separate the words from the phrase
w2v_df = get_w2vdf(df, 'produto')

In [ ]:
w2v_df

**Treinamento**


In [5]:
model_name = "w2v_v2w7s300e750"
model_name

'w2v_v2w7s300e750'

In [ ]:
# train the model using the parameters above
# after training save the model for pre-load
w2v_model = train_w2v(w2v_df)
print("-- Saving the model\n")

# w2v_model.save("word2vec.model")

w2v_model.save(model_name+".model")

**Carrega o modelo pre-treinado**

In [6]:
# load the pre-trained model
# w2v_model = Word2Vec.load("word2vec.model")
# w2v_model = Word2Vec.load("word2vec_v2.model")
w2v_model = Word2Vec.load(model_name+".model")

**Removing infrequent words**

In [153]:
# remove the words OOV (out of vocabulary) from the given text list
def remove_infrequent_words(dataframe):
    counter = 0
    for i in range(len(dataframe)):
        # need to put >list< because i'm modifying a list while iterating over it
        for word in list(dataframe[i]):
            if not word in w2v_model.wv.key_to_index:
                counter += 1
                dataframe[i].remove(word)

    print("removed words: ",counter)
    return dataframe

In [154]:
w2v_df = remove_infrequent_words(w2v_df)

removed words:  34319


**Calcula o Data Frequency (DF)**

In [155]:
## tentativa 04
## calcula o DF (data frequency) do dataframe
data_freq = {}

w2v_df_len = len(w2v_df)

for i in range(w2v_df_len):
    tokens = w2v_df[i]
    for w in tokens:
        try:
            data_freq[w].add(i)
        except:
            data_freq[w] = {i}
            
for i in data_freq:
    data_freq[i] = len(data_freq[i])

In [156]:
def doc_freq(word):
    c = 0
    try:
        c = data_freq[word]
    except:
        pass
    return c

**Calcula o IDF**

In [157]:
idf = {}

for i in range(w2v_df_len):
    tokens = w2v_df[i]
    for token in np.unique(tokens):
        token_df = doc_freq(token)
        calc_idf = np.log10((w2v_df_len + 1) / (token_df + 1))

        idf[token] = calc_idf

**Funções de Similaridade**

In [123]:
def similarity_cosine(vec1, vec2):
    cosine_similarity = np.dot(matutils.unitvec(vec1), matutils.unitvec(vec2))
    return cosine_similarity

In [81]:
# creates a representation of the sentence from the
# average of the sentence's word vectors 
def average_sentence2vec(w2v_df):
    flag = True
    vector = 0
    for word in w2v_df:
        if(flag):
            vector = w2v_model.wv[word]
            flag = False
        else:
            vector = np.add(vector, w2v_model.wv[word])
    vector = vector / (len(w2v_df) + 1)
    return vector        

In [82]:
# creates a representation of the sentence from the
# sum of the sentence's word vectors 
def sum_sentence2vec(w2v_df):
    flag = True
    vector = 0
    for word in w2v_df:
        if(flag):
            vector = w2v_model.wv[word]
            flag = False
        else:
            vector = np.add(vector, w2v_model.wv[word])
    return vector        

In [83]:
# creates a representation of the sentence from the
# sum of the sentence's word vectors 
def weighted_sentence2vec(w2v_df):
    flag = True
    vector = 0
    for word in w2v_df:
        if(flag):
            vector = w2v_model.wv[word] * idf[word]
            flag = False
        else:
            vector = np.add(vector, w2v_model.wv[word] * idf[word])
    vector = vector / (len(w2v_df) + 1) ##verificar
    return vector        

In [84]:
# calc the similarity between two sentences using cos
def average_sentence_similarity(vector1, vector2):
    try:
        similarity = np.dot(matutils.unitvec(vector1), matutils.unitvec(vector2))
    except:
        similarity = 0
    return similarity

In [85]:
# calc the similarity between two sentences using cos
def sum_sentence_similarity(vector1, vector2):
    try:
        similarity = np.dot(matutils.unitvec(vector1), matutils.unitvec(vector2))
    except:
        similarity = 0
    return similarity

In [86]:
def weighted_sentence_similarity(vector1, vector2):
    try:
        similarity = np.dot(matutils.unitvec(vector1), matutils.unitvec(vector2))
    except:
        similarity = 0
    return similarity

**Importando dados: NFE e Base**

In [87]:
cols = ['produto', 'ncm']
# cols2 = ['produto' 'ncm']
df_base = pd.read_csv('C:/Users/higor/OneDrive/Área de Trabalho/db_tcc/cotri-nome-ncm.csv', header = 0, 
names = cols, engine = 'python', encoding = 'utf-8')
df_nfe = pd.read_csv('C:/Users/higor/OneDrive/Área de Trabalho/db_tcc/nfe-nome-ncm.csv', header = 0, 
names = cols, engine = 'python', encoding = 'utf-8')

**Limpeza: NFE e Base**

In [88]:
# transform the text to string type, lower case, clean data
# and remove stop words

column_name = 'produto'
# dataframe base
df_base['ncm'] = df_base['ncm'].replace('UPDT1', 0)
df_base['ncm'] = df_base['ncm'].fillna(0)
df_base['ncm'] = df_base['ncm'].astype(int)
df_base['produto_base'] = df_base[column_name]
df_base[column_name] = df_base[column_name].astype(str)
df_base[column_name] = df_base[column_name].apply(change_lower)
df_base[column_name] = df_base[column_name].apply(clean_data)
df_base[column_name] = df_base[column_name].apply(remover)

# dataframe nfe
df_nfe['ncm'] = df_nfe['ncm'].astype(int)
df_nfe['produto_nfe'] = df_nfe[column_name]
df_nfe[column_name] = df_nfe[column_name].astype(str)
df_nfe[column_name] = df_nfe[column_name].apply(change_lower)
df_nfe[column_name] = df_nfe[column_name].apply(clean_data)
df_nfe[column_name] = df_nfe[column_name].apply(remover)

# separate the words from the phrase
w2v_df_base = get_w2vdf(df_base, column_name)

w2v_df_nfe = get_w2vdf(df_nfe, column_name)

## removing infrequent words in the dataframe base and nfe

w2v_df_base = remove_infrequent_words(w2v_df_base)

w2v_df_nfe = remove_infrequent_words(w2v_df_nfe)

removed words:  33685
removed words:  573


**Appending the dataframes**

In [89]:
# append the w2v dataframe as a column in the df

df_base['w2v'] = w2v_df_base

df_nfe['w2v'] = w2v_df_nfe

**Criando o Dataframe**

In [90]:
columns_db_final = ['produto_base', 'produto_nfe', 'w2v_base', 'w2v_nfe', 'ncm', 'score']
db_final = pd.DataFrame(columns=columns_db_final)

**Most Similar - w2v average**

In [25]:
file_name = "a_"+model_name

In [111]:
db_final = pd.DataFrame(columns=columns_db_final)
for i in range(len(df_nfe)):
    
    score_final = 0
    score = 0
    first_element = True
    idx = 0

    ncm = df_nfe.loc[i, 'ncm']
    df_temp = df_base.loc[df_base['ncm'] == ncm]

    if len(df_temp) > 0:

        for j in range(len(df_temp)):

            if first_element:
                score_final = average_sentence_similarity(
                    average_sentence2vec(df_nfe.loc[i, 'w2v']), 
                    average_sentence2vec(df_temp.loc[df_temp.index[j], 'w2v']))
                idx = df_temp.index[j]
                first_element = False
            else:
                score = average_sentence_similarity(
                    average_sentence2vec(df_nfe.loc[i, 'w2v']), 
                    average_sentence2vec(df_temp.loc[df_temp.index[j], 'w2v']))

            if score > score_final:
                score_final = score
                idx = df_temp.index[j]

        # saving the best values in the dataframe
        try:
            db_final.loc[len(db_final.index)] = [
                df_temp.loc[idx, 'produto_base'],
                df_nfe.loc[i, 'produto_nfe'],
                df_temp.loc[idx, 'w2v'],
                df_nfe.loc[i, 'w2v'],
                df_nfe.loc[i, 'ncm'],
                score_final
            ]
        except:
            print(f"something goes wrong: {df_temp.loc[idx,]}\n{df_nfe.loc[i,]}")
    
    else:
        print(f'product with 0 similars: {df_nfe.loc[i,]}\n')

db_final.to_csv('results/'+file_name+'.csv', encoding='utf-8')

product with 0 similars: produto            saco pap 15kg 32x55   mil
ncm                                 48043190
produto_nfe        SACO PAP 15Kg 32x55 / MIL
w2v            [saco, pap, 15kg, 32x55, mil]
Name: 1358, dtype: object



**Most Similar - w2v weighted**

In [136]:
file_name = "w_"+model_name


In [137]:
db_final = pd.DataFrame(columns=columns_db_final)
for i in range(len(df_nfe)):
    
    score_final = 0
    score = 0
    first_element = True
    idx = 0

    ncm = df_nfe.loc[i, 'ncm']
    df_temp = df_base.loc[df_base['ncm'] == ncm]

    if len(df_temp) > 0:

        for j in range(len(df_temp)):

            if first_element:
                score_final = weighted_sentence_similarity(
                    weighted_sentence2vec(df_nfe.loc[i, 'w2v']), 
                    weighted_sentence2vec(df_temp.loc[df_temp.index[j], 'w2v']))
                idx = df_temp.index[j]
                first_element = False
            else:
                score = weighted_sentence_similarity(
                    weighted_sentence2vec(df_nfe.loc[i, 'w2v']), 
                    weighted_sentence2vec(df_temp.loc[df_temp.index[j], 'w2v']))

            if score > score_final:
                score_final = score
                idx = df_temp.index[j]

        # saving the best values in the dataframe
        try:
            db_final.loc[len(db_final.index)] = [
                df_temp.loc[idx, 'produto_base'],
                df_nfe.loc[i, 'produto_nfe'],
                df_temp.loc[idx, 'w2v'],
                df_nfe.loc[i, 'w2v'],
                df_nfe.loc[i, 'ncm'],
                score_final
            ]
        except:
            print(f"something goes wrong: {df_temp.loc[idx,]}\n{df_nfe.loc[i,]}")
    
    else:
        print(f'product with 0 similars: {df_nfe.loc[i,]}\n')

db_final.to_csv('results/'+file_name+'.csv', encoding='utf-8')

product with 0 similars: produto            saco pap 15kg 32x55   mil
ncm                                 48043190
produto_nfe        SACO PAP 15Kg 32x55 / MIL
w2v            [saco, pap, 15kg, 32x55, mil]
Name: 1358, dtype: object



**Most Similar - w2v Sum**

In [49]:
file_name = "s_"+model_name

In [50]:
db_final = pd.DataFrame(columns=columns_db_final)
for i in range(len(df_nfe)):
    
    score_final = 0
    score = 0
    first_element = True
    idx = 0

    ncm = df_nfe.loc[i, 'ncm']
    df_temp = df_base.loc[df_base['ncm'] == ncm]

    if len(df_temp) > 0:

        for j in range(len(df_temp)):

            if first_element:
                score_final = sum_sentence_similarity(
                    sum_sentence2vec(df_nfe.loc[i, 'w2v']), 
                    sum_sentence2vec(df_temp.loc[df_temp.index[j], 'w2v']))
                idx = df_temp.index[j]
                first_element = False
            else:
                score = sum_sentence_similarity(
                    sum_sentence2vec(df_nfe.loc[i, 'w2v']), 
                    sum_sentence2vec(df_temp.loc[df_temp.index[j], 'w2v']))

            if score > score_final:
                score_final = score
                idx = df_temp.index[j]

        # saving the best values in the dataframe
        try:
            db_final.loc[len(db_final.index)] = [
                df_temp.loc[idx, 'produto_base'],
                df_nfe.loc[i, 'produto_nfe'],
                df_temp.loc[idx, 'w2v'],
                df_nfe.loc[i, 'w2v'],
                df_nfe.loc[i, 'ncm'],
                score_final
            ]
        except:
            print(f"something goes wrong: {df_temp.loc[idx,]}\n{df_nfe.loc[i,]}")
    
    else:
        print(f'product with 0 similars: {df_nfe.loc[i,]}\n')

db_final.to_csv('results/'+file_name+'.csv', encoding='utf-8')

product with 0 similars: produto            saco pap 15kg 32x55   mil
ncm                                 48043190
produto_nfe        SACO PAP 15Kg 32x55 / MIL
w2v            [saco, pap, 15kg, 32x55, mil]
Name: 1358, dtype: object



**Most Similar - w2v average + func**

In [97]:
file_name = "a_jarowink_"+model_name
file_name

'a_jarowink_w2v_v2w7s300e750'

In [96]:
textdistance.JaroWinkler().similarity('soccer', 'socrr')

0.8755555555555556

In [98]:
db_final = pd.DataFrame(columns=columns_db_final)
for i in range(len(df_nfe)):
    
    score_final = 0
    score = 0
    score_w2v = 0
    score_similarity = 0

    first_element = True
    idx = 0

    ncm = df_nfe.loc[i, 'ncm']
    df_temp = df_base.loc[df_base['ncm'] == ncm]

    if len(df_temp) > 0:

        for j in range(len(df_temp)):

            if first_element:
                
                score_w2v = average_sentence_similarity(
                    average_sentence2vec(df_nfe.loc[i, 'w2v']), 
                    average_sentence2vec(df_temp.loc[df_temp.index[j], 'w2v']))

                score_similarity = textdistance.JaroWinkler().similarity(df_nfe.loc[i, 'produto'],
                df_temp.loc[df_temp.index[j], 'produto'])

                if score_similarity == 0:
                    score_similarity = 0.0001

                score_final = 2 * ((score_w2v * score_similarity) / (score_w2v + score_similarity))

                idx = df_temp.index[j]
                first_element = False
            else:
                score_w2v = average_sentence_similarity(
                    average_sentence2vec(df_nfe.loc[i, 'w2v']), 
                    average_sentence2vec(df_temp.loc[df_temp.index[j], 'w2v']))

                score_similarity = textdistance.JaroWinkler().similarity(df_nfe.loc[i, 'produto'],
                df_temp.loc[df_temp.index[j], 'produto'])

                if score_similarity == 0:
                    score_similarity = 0.0001

                score = 2 * ((score_w2v * score_similarity) / (score_w2v + score_similarity))

            if score > score_final:
                score_final = score
                idx = df_temp.index[j]

        # saving the best values in the dataframe
        try:
            db_final.loc[len(db_final.index)] = [
                df_temp.loc[idx, 'produto_base'],
                df_nfe.loc[i, 'produto_nfe'],
                df_temp.loc[idx, 'w2v'],
                df_nfe.loc[i, 'w2v'],
                df_nfe.loc[i, 'ncm'],
                score_final
            ]
        except:
            print(f"something goes wrong: {df_temp.loc[idx,]}\n{df_nfe.loc[i,]}")
    
    else:
        print(f'product with 0 similars: {df_nfe.loc[i,]}\n')

db_final.to_csv('results/'+file_name+'.csv', encoding='utf-8')

product with 0 similars: produto            saco pap 15kg 32x55   mil
ncm                                 48043190
produto_nfe        SACO PAP 15Kg 32x55 / MIL
w2v            [saco, pap, 15kg, 32x55, mil]
Name: 1358, dtype: object



**Most Similar - w2v weight + func**

In [137]:
file_name = "w_jarowink_"+model_name
file_name

'w_jarowink_w2v_v2w5s300e500'

In [138]:
db_final = pd.DataFrame(columns=columns_db_final)
for i in range(len(df_nfe)):
    
    score_final = 0
    score = 0
    score_w2v = 0
    score_similarity = 0

    first_element = True
    idx = 0

    ncm = df_nfe.loc[i, 'ncm']
    df_temp = df_base.loc[df_base['ncm'] == ncm]

    if len(df_temp) > 0:

        for j in range(len(df_temp)):

            if first_element:
                
                score_w2v = weighted_sentence_similarity(
                    weighted_sentence2vec(df_nfe.loc[i, 'w2v']), 
                    weighted_sentence2vec(df_temp.loc[df_temp.index[j], 'w2v']))

                score_similarity = textdistance.JaroWinkler().similarity(df_nfe.loc[i, 'produto'],
                df_temp.loc[df_temp.index[j], 'produto'])

                if score_similarity == 0:
                    score_similarity = 0.0001

                score_final = 2 * ((score_w2v * score_similarity) / (score_w2v + score_similarity))

                idx = df_temp.index[j]
                first_element = False
            else:
                score_w2v = weighted_sentence_similarity(
                    weighted_sentence2vec(df_nfe.loc[i, 'w2v']), 
                    weighted_sentence2vec(df_temp.loc[df_temp.index[j], 'w2v']))

                score_similarity = textdistance.JaroWinkler().similarity(df_nfe.loc[i, 'produto'],
                df_temp.loc[df_temp.index[j], 'produto'])

                if score_similarity == 0:
                    score_similarity = 0.0001

                score = 2 * ((score_w2v * score_similarity) / (score_w2v + score_similarity))

            if score > score_final:
                score_final = score
                idx = df_temp.index[j]

        # saving the best values in the dataframe
        try:
            db_final.loc[len(db_final.index)] = [
                df_temp.loc[idx, 'produto_base'],
                df_nfe.loc[i, 'produto_nfe'],
                df_temp.loc[idx, 'w2v'],
                df_nfe.loc[i, 'w2v'],
                df_nfe.loc[i, 'ncm'],
                score_final
            ]
        except:
            print(f"something goes wrong: {df_temp.loc[idx,]}\n{df_nfe.loc[i,]}")
    
    else:
        print(f'product with 0 similars: {df_nfe.loc[i,]}\n')

db_final.to_csv('results/'+file_name+'.csv', encoding='utf-8')

product with 0 similars: produto            saco pap 15kg 32x55   mil
ncm                                 48043190
produto_nfe        SACO PAP 15Kg 32x55 / MIL
w2v            [saco, pap, 15kg, 32x55, mil]
Name: 1358, dtype: object



**Gabarito - Importação**

In [26]:
# gabarito da base de dados v1
cols_gab = ['id', 'produto_nfe', 'produto_cotriba']
# df_gabarito = pd.read_csv('C:/Users/higor/OneDrive/Área de Trabalho/word2vec/gabarito.csv', header = 0, 
# names = cols_gab, engine = 'python', encoding = 'utf-8')

# gabarito da base de dados com mais correções v2
df_gabarito = pd.read_csv('C:/Users/higor/OneDrive/Área de Trabalho/word2vec/gabarito_c2.csv', header = 0, 
names = cols_gab, engine = 'python', encoding = 'utf-8')

In [27]:
df_gabarito['produto_nfe'] = df_gabarito['produto_nfe'].astype(str)
# df_gabarito['produto_nfe'] = df_gabarito['produto_nfe'].apply(change_lower)

df_gabarito['produto_cotriba'] = df_gabarito['produto_cotriba'].astype(str)
# df_gabarito['produto_cotriba'] = df_gabarito['produto_cotriba'].apply(change_lower)

**dataframe Average**

In [112]:
# dataframe average v2
cols_dfw = ['produto_base', 'produto_nfe', 'w2v_base', 'w2v_nfe', 'ncm', 'score']

df_average = pd.read_csv('C:/Users/higor/OneDrive/Área de Trabalho/word2vec/results/'+file_name+'.csv', header = 0, 
names = cols_dfw, engine = 'python', encoding = 'utf-8')

df_len = len(df_average)

In [31]:
cols_dfw = ['produto_base', 'produto_nfe', 'w2v_base', 'w2v_nfe', 'ncm', 'score']
df_average = db_final
df_len = len(df_average)

In [ ]:
df_average

**Average - F1**

In [113]:
count_total = 0
count_find = 0
for i in range(len(df_average)):
    nfe_equal = 0
    gabarito = False
    equivalente = df_average.loc[i]
    nfe_equal = df_gabarito.loc[df_gabarito['produto_nfe'] == equivalente['produto_nfe']]

    if (len(nfe_equal) > 0):
        gabarito = df_gabarito.loc[nfe_equal.index,'produto_cotriba'].values == equivalente['produto_base']

    if(len(nfe_equal) > 0 and (gabarito.all() == True)):
        count_find += 1
        count_total += 1
    elif (len(nfe_equal) > 0 and (gabarito.all() == False)):
        count_total += 1

print(f'recuperados: {count_find}')
print(f'encontrados: {count_total}')
print(f'total: {df_len}')

recuperados: 8369
encontrados: 9651
total: 9720


In [114]:
precisao = count_find / count_total
print(precisao)

0.8671640244534246


In [115]:
revocacao = count_find / df_len
print(revocacao)

0.8610082304526749


In [ ]:
f1 = 2 * ( (precisao * revocacao) / (precisao + revocacao) )
print(f1)

0.8640751639048061


**dataframe Weighted**

In [139]:
cols_dfw = ['produto_base', 'produto_nfe', 'w2v_base', 'w2v_nfe', 'ncm', 'score']

# dataframe weighted v1
# df_weighted = pd.read_csv('C:/Users/higor/OneDrive/Área de Trabalho/word2vec/results/word2vec_weighted.csv', header = 0, 
# names = cols_dfw, engine = 'python', encoding = 'utf-8')

# dataframe weighted v2
# df_weighted = pd.read_csv('C:/Users/higor/OneDrive/Área de Trabalho/word2vec/results/word2vec_weighted2.csv', header = 0, 
# names = cols_dfw, engine = 'python', encoding = 'utf-8')

# TESTE
df_weighted = pd.read_csv('C:/Users/higor/OneDrive/Área de Trabalho/word2vec/results/'+file_name+'.csv', header = 0, 
names = cols_dfw, engine = 'python', encoding = 'utf-8')

df_len = len(df_weighted)

In [30]:
cols_dfw = ['produto_base', 'produto_nfe', 'w2v_base', 'w2v_nfe', 'ncm', 'score']
df_weighted = db_final
df_len = len(df_weighted)

**Weighted - F1**

In [140]:
columns_weighted_gabarito = ['produto_base_w2v', 'produto_nfe_w2v',
 'produto_base_gab', 'produto_nfe_gab', 
 'w2v_base', 'w2v_nfe', 'ncm', 'score']
df_weighted_gabarito = pd.DataFrame(columns=columns_weighted_gabarito)

In [141]:
count_total = 0
count_find = 0
for i in range(len(df_weighted)):
    nfe_equal = 0
    gabarito = False
    equivalente = df_weighted.loc[i]
    nfe_equal = df_gabarito.loc[df_gabarito['produto_nfe'] == equivalente['produto_nfe']]

    if (len(nfe_equal) > 0):
        gabarito = df_gabarito.loc[nfe_equal.index,'produto_cotriba'].values == equivalente['produto_base']

    if(len(nfe_equal) > 0 and (gabarito.all() == True)):
        count_find += 1
        count_total += 1
    elif (len(nfe_equal) > 0 and (gabarito.all() == False)):
        count_total += 1
        df_weighted_gabarito.loc[len(df_weighted_gabarito.index)] = [
                equivalente['produto_base'],
                equivalente['produto_nfe'],
                df_gabarito.loc[nfe_equal.index,'produto_cotriba'].values,
                df_gabarito.loc[nfe_equal.index,'produto_nfe'].values,
                equivalente['w2v_base'],
                equivalente['w2v_nfe'],
                equivalente['ncm'],
                equivalente['score'],
            ]

print(f'recuperados: {count_find}')
print(f'encontrados: {count_total}')
print(f'total: {df_len}')

    

recuperados: 8329
encontrados: 9651
total: 9720


In [ ]:
df_weighted_gabarito.to_csv('results/word2vec_weighted_gabarito4.csv', encoding='utf-8')

In [142]:
precisao = count_find / count_total
print(precisao)

0.8630193762304424


In [143]:
revocacao = count_find / df_len
print(revocacao)

0.8568930041152263


In [144]:
f1 = 2 * ( (precisao * revocacao) / (precisao + revocacao) )
print(f1)

0.8599452790253471


**dataframe Summed**

In [51]:
# dataframe summed v2
cols_dfw = ['produto_base', 'produto_nfe', 'w2v_base', 'w2v_nfe', 'ncm', 'score']
df_summed = pd.read_csv('C:/Users/higor/OneDrive/Área de Trabalho/word2vec/results/'+file_name+'.csv', header = 0, 
names = cols_dfw, engine = 'python', encoding = 'utf-8')

df_len = len(df_summed)

In [39]:
cols_dfw = ['produto_base', 'produto_nfe', 'w2v_base', 'w2v_nfe', 'ncm', 'score']
df_summed = db_final
df_len = len(df_summed)

**Summed - F1**

In [52]:
count_total = 0
count_find = 0
for i in range(len(df_summed)):
    nfe_equal = 0
    gabarito = False
    equivalente = df_summed.loc[i]
    nfe_equal = df_gabarito.loc[df_gabarito['produto_nfe'] == equivalente['produto_nfe']]

    if (len(nfe_equal) > 0):
        gabarito = df_gabarito.loc[nfe_equal.index,'produto_cotriba'].values == equivalente['produto_base']

    if(len(nfe_equal) > 0 and (gabarito.all() == True)):
        count_find += 1
        count_total += 1
    elif (len(nfe_equal) > 0 and (gabarito.all() == False)):
        count_total += 1

print(f'recuperados: {count_find}')
print(f'encontrados: {count_total}')
print(f'total: {df_len}')

recuperados: 8355
encontrados: 9651
total: 9720


In [53]:
precisao = count_find / count_total
print(precisao)

0.8657133975753808


In [54]:
revocacao = count_find / df_len
print(revocacao)

0.8595679012345679


In [55]:
f1 = 2 * ( (precisao * revocacao) / (precisao + revocacao) )
print(f1)

0.8626297041969956


**dataframe Average + func**

In [164]:
teste_name = 'a_cosine_'+model_name
teste_name

'a_cosine_w2v_v2w7s300e750'

In [165]:
# dataframe average + func
cols_dfw = ['produto_base', 'produto_nfe', 'w2v_base', 'w2v_nfe', 'ncm', 'score']

df_average_func = pd.read_csv('C:/Users/higor/OneDrive/Área de Trabalho/word2vec/results/'+teste_name+'.csv', header = 0, 
names = cols_dfw, engine = 'python', encoding = 'utf-8')

df_len = len(df_average_func)

In [139]:
file_name

'w_jarowink_w2v_v2w5s300e500'

In [ ]:
df_average_func

In [166]:
count_total = 0
count_find = 0
for i in range(len(df_average_func)):
    nfe_equal = 0
    gabarito = False
    equivalente = df_average_func.loc[i]
    nfe_equal = df_gabarito.loc[df_gabarito['produto_nfe'] == equivalente['produto_nfe']]

    if (len(nfe_equal) > 0):
        gabarito = df_gabarito.loc[nfe_equal.index,'produto_cotriba'].values == equivalente['produto_base']

    if(len(nfe_equal) > 0 and (gabarito.all() == True)):
        count_find += 1
        count_total += 1
    elif (len(nfe_equal) > 0 and (gabarito.all() == False)):
        count_total += 1

print(f'recuperados: {count_find}')
print(f'encontrados: {count_total}')
print(f'total: {df_len}')

recuperados: 8487
encontrados: 9651
total: 9720


In [167]:
precisao = count_find / count_total
print(precisao)

0.8793907367112216


In [168]:
revocacao = count_find / df_len
print(revocacao)

0.8731481481481481


In [169]:
f1 = 2 * ( (precisao * revocacao) / (precisao + revocacao) )
print(f1)

0.8762583242992101


**Cosine - textual**

In [21]:
cols = ['produto', 'ncm']
# cols2 = ['produto' 'ncm']
df_base = pd.read_csv('C:/Users/higor/OneDrive/Área de Trabalho/db_tcc/cotri-nome-ncm.csv', header = 0, 
names = cols, engine = 'python', encoding = 'utf-8')
df_nfe = pd.read_csv('C:/Users/higor/OneDrive/Área de Trabalho/db_tcc/nfe-nome-ncm.csv', header = 0, 
names = cols, engine = 'python', encoding = 'utf-8')

**Pré-processamento**

In [22]:
def clean_data_text(text):
    try:
        text = unicode(text, 'utf-8')
    except NameError:
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    text = text.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    text = text.lower()
    text = re.sub('\s\s+', ' ', text.strip())
    return str(text)

In [23]:
# pre processing database
df_base['ncm'] = df_base['ncm'].replace('UPDT1', 0)
df_base['ncm'] = df_base['ncm'].fillna(0)
df_base['ncm'] = df_base['ncm'].astype(int)
df_base['produto_base'] = df_base['produto']
df_base['produto'] = df_base['produto'].apply(clean_data_text)

# pre processing nfe
df_nfe['ncm'] = df_nfe['ncm'].astype(int)
df_nfe['produto_nfe'] = df_nfe['produto']
df_nfe['produto'] = df_nfe['produto'].apply(clean_data_text)

In [97]:
columns_cos = ['produto_base', 'produto_nfe', 'produto_base_pp', 'produto_nfe_pp', 'ncm', 'score']
db_final = pd.DataFrame(columns=columns_cos)

In [98]:
db_final

,produto_base,produto_nfe,produto_base_pp,produto_nfe_pp,ncm,score


In [99]:
for i in range(len(df_nfe)):
    
    score_final = 0
    score = 0
    first_element = True
    idx = 0

    ncm = df_nfe.loc[i, 'ncm']
    df_temp = df_base.loc[df_base['ncm'] == ncm]
    
    if len(df_temp) > 0:

        for j in range(len(df_temp)):

            if first_element:
                score_final = textdistance.Overlap().similarity(df_nfe.loc[i, 'produto'],
                df_temp.loc[df_temp.index[j], 'produto'])
                idx = df_temp.index[j]
                first_element = False
            else:
                score = textdistance.Overlap().similarity(df_nfe.loc[i, 'produto'],
                df_temp.loc[df_temp.index[j], 'produto'])
                
            if score > score_final:
                score_final = score
                idx = df_temp.index[j]

        # saving the best values in the dataframe
        try:
            # db_final.loc[len(db_final.index)] = [
            #     df_temp.loc[idx, 'produto'],
            #     df_nfe.loc[i, 'produto'],
            #     df_nfe.loc[i, 'ncm'],
            #     score_final
            # ]

            db_final.loc[len(db_final.index)] = [
                df_temp.loc[idx, 'produto_base'],
                df_nfe.loc[i, 'produto_nfe'],
                df_temp.loc[idx, 'produto'],
                df_nfe.loc[i, 'produto'],
                df_nfe.loc[i, 'ncm'],
                score_final
            ]
        except Exception as e:
            print(str(e))
            print(f"something goes wrong: {df_temp.loc[idx,'produto_base']}\n{df_nfe.loc[i,'produto_nfe']}")
    
    else:
        print(f'product with 0 similars: {df_nfe.loc[i,]}\n')

db_final.to_csv('results/overlap4.csv', encoding='utf-8')

product with 0 similars: produto          saco pap 15kg 32x55 mil
ncm                             48043190
produto_nfe    SACO PAP 15Kg 32x55 / MIL
Name: 1358, dtype: object



In [ ]:
db_final

In [58]:
# gabarito da base de dados com mais correções v2
cols_gab = ['id', 'produto_nfe', 'produto_cotriba']
df_gabarito = pd.read_csv('C:/Users/higor/OneDrive/Área de Trabalho/word2vec/gabarito_c2.csv', header = 0, 
names = cols_gab, engine = 'python', encoding = 'utf-8')

In [59]:
df_gabarito['produto_nfe'] = df_gabarito['produto_nfe'].astype(str)
# df_gabarito['produto_nfe'] = df_gabarito['produto_nfe'].apply(change_lower)

df_gabarito['produto_cotriba'] = df_gabarito['produto_cotriba'].astype(str)
# df_gabarito['produto_cotriba'] = df_gabarito['produto_cotriba'].apply(change_lower)

In [100]:
# TESTE
co_cosine = ['produto_base', 'produto_nfe', 'ncm', 'score']
co_cosine2 = ['produto_base', 'produto_nfe','produto_base_pp', 'produto_nfe_pp', 'ncm', 'score']

df_cos = pd.read_csv('C:/Users/higor/OneDrive/Área de Trabalho/word2vec/results/overlap4.csv', header = 0, 
names = co_cosine2, engine = 'python', encoding = 'utf-8')

df_len = len(df_cos)

In [101]:
count_total = 0
count_find = 0
for i in range(len(df_cos)):
    nfe_equal = 0
    gabarito = False
    equivalente = df_cos.loc[i]
    nfe_equal = df_gabarito.loc[df_gabarito['produto_nfe'] == equivalente['produto_nfe']]

    if (len(nfe_equal) > 0):
        gabarito = df_gabarito.loc[nfe_equal.index,'produto_cotriba'] == equivalente['produto_base']
    if(len(nfe_equal) > 0 and (gabarito.all() == True)):
        count_find += 1
        count_total += 1
    elif (len(nfe_equal) > 0 and (gabarito.all() == False)):
        count_total += 1

print(f'recuperados: {count_find}')
print(f'encontrados: {count_total}')
print(f'total: {df_len}')


recuperados: 6775
encontrados: 9651
total: 9720


In [102]:
precisao = count_find / count_total
print(precisao)

0.7019997927675888


In [103]:
revocacao = count_find / df_len
print(revocacao)

0.6970164609053497


In [104]:
f1 = 2 * ( (precisao * revocacao) / (precisao + revocacao) )
print(f1)

0.6994992514583656


**Testes**

In [36]:
w2v_model.wv["sabao"].shape

(300,)

In [31]:
w2v_model.wv.most_similar(positive=["pao"])

[('puravi', 0.5243518352508545),
 ('grostolli', 0.5223467946052551),
 ('congelad', 0.5142188668251038),
 ('tickroc', 0.4930357038974762),
 ('sovital', 0.4914597272872925),
 ('grostoli', 0.49016380310058594),
 ('sovado', 0.4899028241634369),
 ('baguete', 0.48934367299079895),
 ('cabritinho', 0.48825597763061523),
 ('cervejinha', 0.48750758171081543)]

In [ ]:
print(sum_sentence2vec(w2v_df[0]))

In [ ]:
print(average_sentence2vec(w2v_df[0]))

In [ ]:
print(weighted_sentence2vec(w2v_df[0]))

In [27]:
average_sentence_similarity(average_sentence2vec(w2v_df[510]), average_sentence2vec(w2v_df[1377]))


0.6614604

In [28]:
sum_sentence_similarity(sum_sentence2vec(w2v_df[510]), sum_sentence2vec(w2v_df[1377]))


0.6614604

In [32]:
weighted_sentence_similarity(weighted_sentence2vec(w2v_df[510]), weighted_sentence2vec(w2v_df[1377]))

0.6152559